Итак, перед тем, как браться за статистику, нужно:

**1. Прочесть исходный файл и превратить его в структуру данных**

К заданию прилагается файл в формате csv, где все значения разделены запятыми. Это наши исходные данные. Чтобы применить к ним все возможности языка Python и библиотеки Pandas, надо импортировать эту библиотеку и сохранить её в переменной. По сокращённому названию панельных данных (panel data), с которых начиналась Pandas, эту переменную принято называть pd:

**import pandas as pd**

Для чтения csv-файла в библиотеке Pandas есть готовая функция — метод **read_csv()**. Как и все методы, он вызывается записью через точку после имени своего объекта. В скобках указывается аргумент (параметр) метода. У read_csv() это имя файла с данными. Прочтение превращает файл в структуру данных DataFrame. Имя переменной, в которой эта структура данных сохраняется, чаще всего df либо отражает тематику данных:

**df = pd.read_csv('music_log.csv')**

**2. Посмотреть на данные**

Вывести на экран таблицу и оценить данные:

print(df)
Как правило, таблица очень велика. Практичнее запросить определённое количество первых строк, методом head().

**3. Оценить качество предподготовки**

Нужно убедиться в том, что данные прошли предподготовку. По крайней мере, не должно быть пропусков и повторов. Пропущенные и неопределённые значения выявляет метод **isna()**, а суммарное количество таких значений — метод **sum()**.

**print(df.isna().sum())**


Повторяющиеся строки — дубликаты — выявляются методом **duplicated()** и подсчитываются тем же sum():

**print(df.duplicated().sum())**
Если возвращаются нули, всё хорошо — данные пригодны для исследования.


## Задача

1. Загрузите таблицу **exoplanet_catalog**, содержащую данные о планетах за пределами Солнечной системы, или экзопланетах. Сформируйте из данной таблицы новый датасет под назанием **exoplanet**, коорая будет содердать стоблцы name, mass, radius, discovered. Проверьте, чтобы название всех столбцов было корректным, при необходмости измените названия.

2. Удалите из получившейся таблица строки с нулевыми значениями.

3. Убедитесь, что таблица не содержит ни пропущенных значений, ни дубликатов.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/exoplanet_catalog.csv')

print(df.head(10))
print(df.columns.tolist())

         # name planet_status  mass  mass_error_min  mass_error_max  \
0      11 Com b     Confirmed   NaN             NaN             NaN   
1      11 Oph b     Confirmed  21.0             3.0             3.0   
2      11 UMi b     Confirmed   NaN             NaN             NaN   
3      14 And b     Confirmed   NaN             NaN             NaN   
4      14 Her b     Confirmed   NaN             NaN             NaN   
5      14 Her c     Confirmed   5.8             1.0             1.4   
6    16 Cyg B b     Confirmed   NaN             NaN             NaN   
7      17 Sco b     Confirmed   NaN             NaN             NaN   
8      18 Del b     Confirmed   NaN             NaN             NaN   
9  1I/'Oumuamua     Confirmed   NaN             NaN             NaN   

   mass_sini  mass_sini_error_min  mass_sini_error_max    radius  \
0      19.40                 1.50                 1.50       NaN   
1        NaN                  NaN                  NaN       NaN   
2      10.50  

In [ ]:
exoplanet = df[['# name', 'planet_status', 'discovered', 'mass', 'radius']] # создание таблицы с нужными столбцами
print(f"Количество строк в сыром датасете: {len(exoplanet)}\n")

exoplanet.columns = ['name', 'planet_status', 'discovered', 'mass', 'radius']

# Удаление строк с пустыми и/или нулевыми значениями
exoplanet = exoplanet.loc[(exoplanet['mass'].notna()) & (exoplanet['mass'] != 0) &
                          (exoplanet['radius'].notna()) & (exoplanet['radius'] != 0) &
                          (exoplanet['discovered'].notna()) &
                          (exoplanet['name'].notna())]

# Удаление дубликатов
exoplanet = exoplanet.drop_duplicates()

# Проверка
print(exoplanet.head(10))
print(f"\nКоличество строк в обновлённом датасете: {len(exoplanet)}")

Количество строк в сыром датасете: 4796

                name planet_status  discovered  mass  radius
10       1RXS 1609 b     Confirmed      2008.0  14.0    1.70
18      2M 0122-24 b     Confirmed      2013.0  20.0    1.00
19      2M 0219-39 b     Confirmed      2015.0  13.9    1.44
22      2M 2140+16 b     Confirmed      2010.0  20.0    0.92
23      2M 2206-20 b     Confirmed      2010.0  30.0    1.30
33         2M1510A a     Confirmed      2020.0  40.0    1.57
34         2M1510A b     Confirmed      2020.0  39.3    1.57
41  2MASS J0348-6022     Confirmed      2021.0  43.0    0.91
42  2MASS J0407+1546     Confirmed      2021.0  67.0    0.97
48  2MASS J1219+3128     Confirmed      2021.0  49.0    0.97

Количество строк в обновлённом датасете: 1055


# Группировка данных

Слово «анализ» означает разбор, рассмотрение с разных сторон. Анализ данных начинают с разделения их на группы по какому-нибудь признаку. Эта операция называется группировка данных. Она помогает изучить материал более подробно, чтобы затем перейти к поиску взаимосвязей между отдельными группами.


Группировка оправданна, если данные чётко делятся по значимому признаку, а полученные группы близки к теме задачи. Например, когда есть данные обо всех покупках в супермаркете, можно смело заниматься группировкой. Так можно установить время наплыва покупателей и решить проблему пиковых нагрузок. Или посчитать средний чек — обычно для магазинов это ключевая метрика.


Стадии группировки хорошо укладываются в словесную формулу **split-apply-combine**:

•	разделить, **split** — разбиение на группы по определённому критерию;

•	применить, **apply** — применение какого-либо метода к каждой группе в отдельности, например, подсчёт численности группы методом count() или суммирование вызовом sum();

•	объединить, **combine** — сведение результатов в новую структуру данных, в зависимости от условий разделения и выполнения метода это бывает DataFrame и Series.

В библиотеке Pandas есть отличные инструменты группировки. Рассмотрим обращение с ними на примере анализа данных о планетах за пределами Солнечной системы, или экзопланетах. Орбитальные обсерватории засекли уже тысячи таких небесных тел. Их выявляют на снимках космических телескопов наши коллеги, аналитики данных. Поищем среди экзопланет похожие на Землю. Возможно, это наши будущие колонии, или там уже обитают разумные существа, с которыми однажды предстоит установить контакт.

DataFrame с данными по нескольким тысячам экзопланет сохранён в переменной **exoplanet**. Посмотрим на первые 30 строк таблицы:


In [ ]:
print(exoplanet.head(30))

                 name planet_status  discovered      mass  radius
10        1RXS 1609 b     Confirmed      2008.0  14.00000  1.7000
18       2M 0122-24 b     Confirmed      2013.0  20.00000  1.0000
19       2M 0219-39 b     Confirmed      2015.0  13.90000  1.4400
22       2M 2140+16 b     Confirmed      2010.0  20.00000  0.9200
23       2M 2206-20 b     Confirmed      2010.0  30.00000  1.3000
33          2M1510A a     Confirmed      2020.0  40.00000  1.5700
34          2M1510A b     Confirmed      2020.0  39.30000  1.5700
41   2MASS J0348-6022     Confirmed      2021.0  43.00000  0.9100
42   2MASS J0407+1546     Confirmed      2021.0  67.00000  0.9700
48   2MASS J1219+3128     Confirmed      2021.0  49.00000  0.9700
63           51 Eri b     Confirmed      2015.0   2.60000  1.1100
64           51 Peg b     Confirmed      1995.0   0.47000  1.9000
68           55 Cnc e     Confirmed      2004.0   0.02703  0.1737
83          AD 3116 b     Confirmed      2017.0  54.20000  1.0200
85        

**Документация**

Столбцы:

•	name: название экзопланеты;

•	mass: масса в массах планеты Юпитер;

•	radius: радиус, пересчитанный в радиусах Земли;

•	discovered: год открытия экзопланеты.

*Источник: каталог экзопланет на портале exoplanet.eu*

На картинке изображен принцип **split-apply-combine** для таблицы с экзопланетами. Посмотрим, как вообще идут дела с поиском экзопланет. Сначала данные делят по группам, где каждая группа — это год. Потом метод **count()** подсчитывает численность каждой группы. В итоге получаем новую структуру данных с группами, где каждая содержит год и число открытых за этот год экзопланет.

![image.png](attachment:image.png)

В Рandas для группировки данных есть метод **groupby()**. Он принимает как аргумент название столбца, по которому нужно группировать. В случае с делением экзопланет по годам открытия:


**print(exoplanet.groupby('discovered'))**

**<pandas.core.groupby.DataFrameGroupBy object at 0x7fc1e1ca3400>**


Применение метода **groupby()** к объекту типа DataFrame приводит к созданию объекта особого типа — **DataFrameGroupBy**. Это сгруппированные данные. Если применить к ним какой-нибудь метод Pandas, они станут новой структурой данных типа **DataFrame** или **Series**.
Подсчитаем сгруппированные по годам экзопланеты методом **count()**:


In [ ]:
print(exoplanet.groupby('discovered'))

Если нужно сравнить наблюдения по одному показателю, метод применяют к **DataFrameGroupBy** с указанием на один столбец. Нас в первую очередь интересует радиус экзопланет: мы ищем другую Землю. Давайте получим таблицу с единственным столбцом 'radius':

**exo_number = exoplanet.groupby('discovered')['radius'].count()**

**print(exo_number)**

In [ ]:
exo_number = exoplanet.groupby('discovered')['radius'].count()

print(exo_number)

discovered
1995.0      1
1996.0      1
1999.0      1
2000.0      2
2001.0      1
2002.0      1
2004.0      7
2005.0      4
2006.0     10
2007.0     19
2008.0     25
2009.0     15
2010.0     57
2011.0     96
2012.0     73
2013.0     96
2014.0    105
2015.0     57
2016.0    106
2017.0     63
2018.0    114
2019.0     71
2020.0     88
2021.0     42
Name: radius, dtype: int64


Получили Series, где по годам открытия расписано количество экзопланет, для которых удалось установить радиус.

Посмотрим, как меняется средний радиус открытых экзопланет год от года. Для этого надо сложить радиусы планет, открытых за определённый год, и поделить на их количество (которое мы уже нашли).

Сумма радиусов считается методом **sum()**:

**exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()**


**print(exo_radius_sum)**



In [ ]:
exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()

print(exo_radius_sum)

discovered
1995.0     1.900000
1996.0     1.060000
1999.0     1.380000
2000.0     2.007000
2001.0     0.921000
2002.0     1.200000
2004.0     6.789700
2005.0     4.789000
2006.0    20.355000
2007.0    24.334600
2008.0    34.329000
2009.0    15.366794
2010.0    56.828660
2011.0    77.967374
2012.0    50.074507
2013.0    69.372100
2014.0    55.268000
2015.0    52.684740
2016.0    97.986000
2017.0    51.597620
2018.0    90.195760
2019.0    62.114410
2020.0    65.119180
2021.0    31.915760
Name: radius, dtype: float64


Очень кстати, что объекты Series можно делить друг на друга. Это позволит нам разделить перечень сумм радиусов на перечень количеств экзопланет без перебора в цикле:


**exo_radius_mean = exo_radius_sum/exo_number**

**print(exo_radius_mean)**


In [ ]:
exo_radius_mean = exo_radius_sum/exo_number

print(exo_radius_mean)

discovered
1995.0    1.900000
1996.0    1.060000
1999.0    1.380000
2000.0    1.003500
2001.0    0.921000
2002.0    1.200000
2004.0    0.969957
2005.0    1.197250
2006.0    2.035500
2007.0    1.280768
2008.0    1.373160
2009.0    1.024453
2010.0    0.996994
2011.0    0.812160
2012.0    0.685952
2013.0    0.722626
2014.0    0.526362
2015.0    0.924294
2016.0    0.924396
2017.0    0.819010
2018.0    0.791191
2019.0    0.874851
2020.0    0.739991
2021.0    0.759899
Name: radius, dtype: float64


Точность наших приборов растёт, и новые экзопланеты по размерам всё ближе к Земле.

А теперь вернемся к анализу датафрейма с музыкльными предпочтениями слушателей Яндекс.Музыки. Идею объединения сервисов Музыка и Радио тестировали на небольшой группе пользователей. Результаты сведены в csv-файл, который вам предстоит изучить. Итог анализа таких данных — это метрики: величины, значения которых отражают пользовательские впечатления. Одна из важнейших — **happiness**. Здесь это среднее время, которое пользователь слушает музыку в течение выбранного периода времени (в нашей задаче — за сутки). Чем дольше пользователь слушает музыку, тем он довольнее. Ваша задача: найти значение happiness и посмотреть, как оно менялось.

Тем же методом **groupby()**, которым мы ищем новую Землю, можно поискать и необыкновенного человека в данных Яндекс.Музыки. Тем более, что без этого не выполнить поставленной менеджером задачи.

Прежде, чем рассчитывать метрику happiness, нужно изучить пользователей, чьё «счастье» мы собираемся оценить. Какие они, эти люди, которые слушают действительно много музыки? Есть ли у них особые предпочтения, или они потребляют всё подряд?


## Задача

1. Меломаны у нас есть. Сейчас узнаем идентификатор **user_id** одного из них. Для этого сгруппируем данные по каждому пользователю, чтобы собрать жанры прослушанных им композиций.

Сгруппируйте DataFrame по столбцу user_id, сохраните полученный результат в переменной genre_grouping.

Посчитайте количество жанров, которые выбрали пользователи, методом count(), указав, что выбираем один столбец genre_name.

Сохраните результат в переменной genre_counting и выведите первые 30 строк этой таблицы.

2. Быть может, те, кто за день слушает больше 50 песен, имеют более широкие предпочтения. Чтобы найти такого, изготовим универсальный инструмент.

Напишите функцию user_genres, которая принимает некую группировку как свой аргумент group. Функция должна перебирать группы, входящие в эту группировку.

В каждой группе два элемента — имя группы с индексом 0 и список значений с индексом 1.

Обнаружив такую группу, в которой список (элемент с индексом 1) содержит более 50 значений, функция возвращает имя группы (значение элемента с индексом 0).

3. Вызовите функцию **user_genres**, как аргумент передайте ей **genre_grouping**. Результат – **user_id** неведомого нам любителя музыки – сохраните в переменной **search_id** и выведите значение на экран.


In [ ]:
df_yandex = pd.read_csv('/content/yandex_music.csv')

print(df_yandex.head(10))
df_yandex.columns = ['user_id', 'track', 'artist', 'genre_name', 'city', 'time', 'day']
genre_grouping = df_yandex.groupby('user_id')

genre_counting = genre_grouping['genre_name'].count()
print(genre_counting.head(30))


# g[0] user_id, g[1] DataFrame
def user_genres(group):
    for g in group:
        if len(g[1]) > 50:
            return g[0]
search_id = user_genres(genre_grouping)

print('user_id', search_id)

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

# Сортировка данных

Поиск необычного в группе — что среди планет, что среди меломанов — это прежде всего поиск чемпионов: объектов с выдающимися показателями по разным статьям. Как всю таблицу, так и отдельные группы изучают, сортируя строки по какому-либо столбцу.

В Pandas для этой операции есть метод **sort_values()**. У него два аргумента:

• **by = 'имя столбца'** — имя столбца, по которому нужно сортировать;

• **ascending:** по умолчанию True. Для сортировки по убыванию установите значение False.

![image.png](attachment:image.png)


Среди экзопланет интересны близкие по размерам к Земле. Есть ли такие? Отсортируем список по радиусу в порядке возрастания. Тогда в голове таблицы окажутся самые малые, на которых гравитация не прижмёт нас к полу.

**print(exoplanet.sort_values(by='radius').head(30))**


In [ ]:
print(exoplanet.sort_values(by='radius').head(30))

              name planet_status  discovered      mass   radius
3207   Kepler-37 b     Confirmed      2013.0  0.010000  0.02600
1744  Kepler-102 b     Confirmed      2014.0  0.013530  0.04200
2221  Kepler-138 b     Confirmed      2014.0  0.000210  0.04700
3611   Kepler-62 c     Confirmed      2013.0  0.013000  0.04800
3323   Kepler-42 d     Confirmed      2011.0  0.003000  0.05100
1745  Kepler-102 c     Confirmed      2014.0  0.009440  0.05200
1473      K2-266 c     Confirmed      2018.0  0.000910  0.06300
3322   Kepler-42 c     Confirmed      2011.0  0.006000  0.06500
3208   Kepler-37 c     Confirmed      2013.0  0.037760  0.06700
4473  TRAPPIST-1 h     Confirmed      2017.0  0.001041  0.06740
1711      KOI-55 b     Confirmed      2011.0  0.001400  0.06800
4469  TRAPPIST-1 d     Confirmed      2016.0  0.001300  0.06890
3321   Kepler-42 b     Confirmed      2011.0  0.009000  0.07000
3300  Kepler-408 b     Confirmed      2014.0  0.015730  0.07300
1798  Kepler-106 b     Confirmed      20

Оказывается, некоторые из уже открытых экзопланет по размерам близки не то что к Земле, но уже и к Луне! Получим список экзопланет с радиусом меньше земного. Смотрите, как логический оператор (здесь это <) задействуется в отборе элементов столбца. Дальше нам этот приём не раз понадобится.


**print(exoplanet[exoplanet['radius'] < 1])**

In [ ]:
print(exoplanet[exoplanet['radius'] < 1])

                  name discovered      mass   radius
22        2M 2140+16 b  Confirmed  20.00000  0.92000
41    2MASS J0348-6022  Confirmed  43.00000  0.91000
42    2MASS J0407+1546  Confirmed  67.00000  0.97000
48    2MASS J1219+3128  Confirmed  49.00000  0.97000
68            55 Cnc e  Confirmed   0.02703  0.17370
...                ...        ...       ...      ...
4712  WISE 1217+16 A b  Confirmed  22.00000  0.96000
4732        Wolf 940 b  Confirmed  26.00000  0.92000
4734           XO-2N b  Confirmed   0.62000  0.97300
4751  ZTF J0038+2030 b  Confirmed  59.30000  0.78300
4783          pi Men c  Confirmed   0.01517  0.16719

[567 rows x 4 columns]


Но и этот список такой длинный, что изучать его лучше по частям. Экзопланеты, близкие по размерам к Земле, за последнее десятилетие открывали нередко. Можно изучать список открытых за каждый год. Например, для 2014 года (вновь обратите внимание на работу логического оператора, теперь это ==):

**print(exoplanet[exoplanet['discovered'] == 2014])**


In [ ]:
print(exoplanet[exoplanet['discovered'] == 2014])

             name planet_status  discovered    mass  radius
342      GU Psc b     Confirmed      2014.0  11.000   1.350
394    HAT-P-49 b     Confirmed      2014.0   1.730   1.413
400    HAT-P-54 b     Confirmed      2014.0   0.760   0.944
427     HATS-15 b     Confirmed      2014.0   2.170   1.105
454      HATS-4 b     Confirmed      2014.0   1.323   1.020
...           ...           ...         ...     ...     ...
4673    WASP-74 b     Confirmed      2014.0   0.826   1.248
4685    WASP-83 b     Confirmed      2014.0   0.300   1.040
4688  WASP-87 A b     Confirmed      2014.0   2.210   1.385
4690    WASP-89 b     Confirmed      2014.0   5.900   1.040
4695  WASP-94 A b     Confirmed      2014.0   0.452   1.720

[105 rows x 5 columns]


А чтобы не тратить время на лишнее, поставим оба условия сразу. Для этого в Pandas есть логический оператор &, подобный оператору and языка Python. Напомним, его смысл на русском языке можно передать словами «и ещё»:

**# экзопланеты меньше Земли __ и ещё __ открытые в 2014 году**

**exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]**

**print(exo_small_14)**



In [ ]:
exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]

print(exo_small_14)

              name planet_status  discovered    mass  radius
400     HAT-P-54 b     Confirmed      2014.0  0.7600   0.944
465       HATS-5 b     Confirmed      2014.0  0.2370   0.912
477       HATS-6 b     Confirmed      2014.0  0.3190   0.998
1164  HIP 116454 b     Confirmed      2014.0  0.0372   0.226
1690    KOI-1257 b     Confirmed      2014.0  1.4500   0.940
...            ...           ...         ...     ...     ...
4039   Kepler-95 b     Confirmed      2014.0  0.0410   0.305
4051   Kepler-96 b     Confirmed      2014.0  0.0270   0.238
4065   Kepler-97 b     Confirmed      2014.0  0.0110   0.132
4077   Kepler-98 b     Confirmed      2014.0  0.0110   0.178
4088   Kepler-99 b     Confirmed      2014.0  0.0190   0.132

[78 rows x 5 columns]


Отсортируем результат в порядке убывания радиуса.

**print(exo_small_14.sort_values(by = 'radius', ascending = False))**


In [ ]:
print(exo_small_14.sort_values(by = 'radius', ascending = False))

              name planet_status  discovered     mass  radius
477       HATS-6 b     Confirmed      2014.0  0.31900   0.998
3329  Kepler-425 b     Confirmed      2014.0  0.25000   0.978
3319  Kepler-419 b     Confirmed      2014.0  2.50000   0.960
400     HAT-P-54 b     Confirmed      2014.0  0.76000   0.944
1690    KOI-1257 b     Confirmed      2014.0  1.45000   0.940
...            ...           ...         ...      ...     ...
3300  Kepler-408 b     Confirmed      2014.0  0.01573   0.073
1798  Kepler-106 b     Confirmed      2014.0  0.01668   0.073
1745  Kepler-102 c     Confirmed      2014.0  0.00944   0.052
2221  Kepler-138 b     Confirmed      2014.0  0.00021   0.047
1744  Kepler-102 b     Confirmed      2014.0  0.01353   0.042

[78 rows x 5 columns]


Самая крупная планета, Kepler 106 d – почти как Земля, вращается вокруг звезды Kepler 106 в созвездии Лебедя. Эта звезда очень похожа на наше Солнце. Правда, до неё 1435 световых лет — далековато. Но, возможно, учёные что-нибудь придумают. А мы пока применим эту технологию к нашему бизнесу, в «приземлённой» задаче.
## Задача

1. Космический телескоп Kepler открыл похожую на Землю планету у похожей на Солнце звезды. А вы в данных Яндекс.Музыки обнаружили меломана с уникальными данными. Он за день послушал больше 50 композиций.
Получите таблицу с прослушанными им треками.

Для этого запросите из структуры данных df строки, отвечающие сразу двум условиям:

1) значение в столбце 'user_id' должно быть равно значению переменной search_id;

2) время прослушивания, т.е. значение в столбце 'total_play_seconds', не должно равняться 0.

Сохраните результат в переменной music_user.

2. Теперь узнаем, сколько времени он слушал музыку каждого жанра.

Сгруппируйте данные таблицы music_user по столбцу 'genre_name' и получите сумму значений столбца 'total_play_seconds'. Сохраните результат в переменной sum_music_user и выведите её значение на экран.

3. Кажется, предпочтения нашего меломана начинают проявляться. Но, возможно, длительность композиций от жанра к жанру сильно различается. Важно знать, сколько треков каждого жанра он включил.

Сгруппируйте данные по столбцу genre_name и посчитайте, сколько значений в столбце genre_name. Сохраните результат в переменной count_music_user и выведите её значение на экран.

4. Чтобы предпочтения были видны сразу, нужно крупнейшие значения расположить наверху. Отсортируйте данные в группировке sum_music_user по убыванию. Внимание: когда применяете метод sort_values() к Series с единственным столбцом, аргумент by указывать не нужно, только порядок сортировки.

Сохраните результат в переменной final_sum и выведите её значение на экран.

5. Теперь то же самое надо сделать с числом прослушанных меломаном композиций. Отсортируйте данные группировки count_music_user по убыванию. Сохраните результат в переменной final_count, значение которой выведите на экран.


In [ ]:
# Фильтрация данных по user_id и времени прослушивания > 0
music_user = df_yandex[(df_yandex['user_id'] == search_id) & (df_yandex['time'] != 0)]

# Группировка по жанру и получение суммы времени прослушивания
sum_music_user = music_user.groupby('genre_name')['time'].sum()

# Группировка по жанру и подсчет количества треков
count_music_user = music_user.groupby('genre_name')['genre_name'].count()

# Сортировка по убыванию суммы времени прослушивания
final_sum = sum_music_user.sort_values(ascending=False)

# Сортировка по убыванию количества треков
final_count = count_music_user.sort_values(ascending=False)

# Вывод результатов
print("Сумма времени прослушивания по жанрам:\n", final_sum)
print("\nКоличество прослушанных треков по жанрам:\n", final_count)

Сумма времени прослушивания по жанрам:
 genre_name
videogame      08:47:34
alternative    08:36:53
Name: time, dtype: object

Количество прослушанных треков по жанрам:
 genre_name
alternative    1
videogame      1
Name: genre_name, dtype: int64


Сколько прослушивают рок в Питере в пн и пт

In [9]:
import pandas as pd

data = pd.read_csv('/content/datasets/yandex_music.csv')

data.columns = data.columns.str.strip()

spb_rock_mon_fri = data[(data['genre'] == 'rock') & (data['City'] == 'Saint-Petersburg') & (data['Day'].isin(['Monday', 'Friday']))]

rock_count_mon_fri = spb_rock_mon_fri['Day'].value_counts()
rock_count_mon_fri

,count
Day,
Friday,629
Monday,616
